<a href="https://colab.research.google.com/github/Vanesssin/Prediccion_Euribor/blob/main/Arquitecturas__LSTM_multi_step_ahead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PATH=####

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
SMALL_SIZE = 8 +10
MEDIUM_SIZE = 10 +10
BIGGER_SIZE = 12 +10

plt.rc('font', size=SMALL_SIZE)   # controls default text sizes 
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title 
plt.rc('axes', labelsize=MEDIUM_SIZE) # fontsize of the x and y labels 
plt.rc('xtick', labelsize=SMALL_SIZE) # fontsize of the tick labels 
plt.rc('ytick', labelsize=SMALL_SIZE) # fontsize of the tick labels 
plt.rc('legend', fontsize=SMALL_SIZE) # legend fontsize 
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title

In [ ]:
### SOLO EJECUTAR EL PRIMER DIA DE LOS BARRIDOS - CUADNO EL PICKEL NO ESTÁ GENERADO
import pickle

#res_barrido={}
#pickle_out = open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle","wb")
#pickle.dump(res_barrido, pickle_out)
#pickle_out.close()

In [ ]:
# Codigo para recuperar el pickle de los resultados
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
print(len(res_barrido.keys()))

# Load and prepare data

Data from: https://www.quandl.com/data/BOF/QS_D_IEUTIO3M-EURIBOR-3-Months-Daily

In [ ]:
import json

with open("/content/drive/My Drive/TFM_VIU/data/QS_D_IEUTIO3M_new.json") as file:
  data=json.load(file)
data_source2= pd.DataFrame(data['dataset']['data'])
data_source2.columns=['Date','value']
data_source2.index=pd.to_datetime(data_source2['Date'].astype(str))
data_source2.sort_index(inplace=True,ascending=True)

In [ ]:
### Check NA data and be sure not weekend timestamp values
def check_delta_time(data_serie, n=5):
  delta_time=[]
  for i in range(5,len(data_serie)):
    delta_time.append((data_serie[i]-data_serie[i-n]).days)
  return np.asarray(delta_time)


data_source2['day_of_week']=data_source2.index.dayofweek.values
check_1= check_delta_time(data_source2.index, n=5)
data_source2['check_diffs']=np.append(np.nan*np.ones(5), check_1)

# Fix the same time period (interpolation and imputation of NA values)
first_day=data_source2.index[0]
last_day=data_source2.index[-1]

print('first recording: ', first_day)
print('last recording: ', last_day)

# new range
new_range = pd.date_range(first_day, last_day, freq='D')

# resample
data_source2=data_source2.reindex(new_range)
#Remove week-end days
data_source2['day_of_week']=data_source2.index.dayofweek.values
data_source2=data_source2[(data_source2['day_of_week']!=5)&(data_source2['day_of_week']!=6)]

#Interpolate values
data_source2['value_interp']=data_source2['value'].interpolate().apply(lambda x: np.round(x,decimals=3))

#Temporal serie to model
data=data_source2['value_interp'].reset_index(drop=True)
data=data.values[:,None]



In [ ]:
#Representar data original (temporal serie)
plt.figure(figsize=(25,8))
plt.plot(data)
plt.show()

In [ ]:
#Escalar variables
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)


In [ ]:
#Representar data_scaled
plt.figure(figsize=(25,8))
plt.plot(data_scaled)
plt.show()

In [ ]:
np.min(data)

# General functions

In [ ]:
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def theil (y_test, y_pred, X_test,PH):
  num= (y_test-y_pred)**2
  den=(y_test-X_test[:,-PH:])**2

  return np.sum(num)/np.sum(den)

#If THEIL = 1, the model has a performance equal to a trivial prediction, 
#if THEIL > 1, the performance is worse than a trivial prediction; 
#and if THEIL < 1, the performance is better than a trivial prediction.

In [ ]:
def pocid(inv_yhat,inv_y):

  return 100*np.sum(np.diff(inv_yhat,1, axis=0).ravel()*np.diff(inv_y,1, axis=0).ravel()<0)/len(np.diff(inv_yhat,1, axis=0).ravel())

#The Percentage of Correct (up/down) Directional Prediction
#POCID, shown in Eq. (9), measures the percentage of accuracy
#relating to the series trend.

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# Parameters definition

In [ ]:
## Definir variables train/val/test
N_FEATURES=1
OPTION='A'#'B'
if OPTION =='A':
  per_train=0.6
  per_val=0.2
  per_test=0.2
else:
  per_train=0.7
  per_val=0.2
  per_test=0.1


# split dataset (train/val) 
per_split=0.75

# define seed
import random
random.seed(44)

---

# Model arquitectures

In [ ]:
## model 1
# define the model
def model_1(input_shape,PH):
 model = Sequential()
 model.add(LSTM(50, return_sequences=False,input_shape=input_shape))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

## model 2
# define the model
def model_2(input_shape,PH):
 model = Sequential()
 model.add(LSTM(100, return_sequences=False,input_shape=input_shape))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

## model 3
# define the model
def model_3(input_shape,PH):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(100, return_sequences=False,input_shape=input_shape,dropout=dropout))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model


### Using BLSTM
## model 4
# define the model
def model_4(input_shape,PH):
 dropout = 0.1
 model = Sequential()
 model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=input_shape))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

### Using Stacked LSTM
## model 5
# define the model
def model_5(input_shape,PH):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(70, return_sequences=True,input_shape=input_shape))
 model.add(LSTM(30, return_sequences=False))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

### Using Stacked LSTM
## model 6
# define the model
def model_6(input_shape,PH):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(100, return_sequences=True,input_shape=input_shape))
 model.add(LSTM(50, return_sequences=False))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

 ### Using Stacked LSTM
## model 7
# define the model
def model_7(input_shape,PH):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(70, return_sequences=True,input_shape=input_shape))
 model.add(LSTM(70, return_sequences=False))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

In [ ]:
 ### Using Stacked LSTM
## model 10
# define the model
def model_10(input_shape,PH):
  model = Sequential()
  model.add(LSTM(64, return_sequences=True,input_shape=input_shape))
  model.add(LSTM(64, return_sequences=False))
  model.add(Dense(32))
  model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

 ## model 11
# define the model
def model_11(input_shape,PH):
  dropout = 0.1
  model = Sequential()
  model.add(LSTM(64, return_sequences=True,input_shape=input_shape, dropout=0.2))
  model.add(LSTM(64, return_sequences=False))
  model.add(Dense(32))
  model.add(Dropout(0.2))
  model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

 ## model 12
# define the model
def model_12(input_shape,PH):
  dropout = 0.1
  model = Sequential()
  model.add(LSTM(64, return_sequences=False,input_shape=input_shape))
  model.add(Dense(32))
  model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

 ## model 13
# define the model
def model_13(input_shape,PH):
  dropout = 0.1
  model = Sequential()
  model.add(LSTM(64, return_sequences=False,input_shape=input_shape, dropout=0.2))
  model.add(Dense(32))
  model.add(Dropout(0.2))
  model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

In [ ]:
 ### Using Stacked LSTM
## model 10b
# define the model
def model_10b(input_shape,PH):
  model = Sequential()
  model.add(LSTM(64, return_sequences=True,input_shape=input_shape))
  model.add(LSTM(64, return_sequences=False))
  model.add(Dense(32))
  #model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

 ## model 11b
# define the model
def model_11b(input_shape,PH):
  dropout = 0.1
  model = Sequential()
  model.add(LSTM(64, return_sequences=True,input_shape=input_shape, dropout=0.2))
  model.add(LSTM(64, return_sequences=False))
  model.add(Dense(32))
  model.add(Dropout(0.2))
  #model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

 ## model 12b
# define the model
def model_12b(input_shape,PH):
  dropout = 0.1
  model = Sequential()
  model.add(LSTM(64, return_sequences=False,input_shape=input_shape))
  model.add(Dense(32))
  #model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

 ## model 13b
# define the model
def model_13b(input_shape,PH):
  dropout = 0.1
  model = Sequential()
  model.add(LSTM(64, return_sequences=False,input_shape=input_shape, dropout=0.2))
  model.add(Dense(32))
  model.add(Dropout(0.2))
  #model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

# Barrido N_LAGS modelos

In [ ]:
## BARRIDO DE PARÁMETROS

LIST_PH=[3,7,15]
LIST_N_LAGS=[180,90,30,15]
LIST_N_LAGS=LIST_N_LAGS[1:]

# PREDICTION PARAMS (no- change!)
#PH= 3
#N_LAGS= 180#30
MAX_EPOCHS = 20
BATCH_SIZE=32
PATIENCE=2
OPTIMIZER='adam'
lOSS_METRIC='mae'

In [ ]:
#Cargar librerías
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf

In [ ]:
## mAIN PROCESS

def main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS):
  #Prepar data k-validation
  # PREDICTION PARAMS
  OUT_STEPS=PH
  num_features=N_FEATURES
  #

  #porcessed signal
  X, y = split_sequence(list(data_scaled), N_LAGS, PH)
  data_transf=series_to_supervised(data_scaled, n_in=N_LAGS, n_out=PH, dropnan=True)

  ## Prepare data for 4-fold validation (method 1)

  # division train/val/test
  len_h=len(X)
  dict_kfold={'4': {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)} }
  print(len_h)
  len_h=len_h-len(dict_kfold['4']['Test'])
  dict_kfold ['3']= {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)}
  print(len_h)
  len_h=len_h-len(dict_kfold['3']['Test'])
  dict_kfold['2']= {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)}
  print(len_h)
  len_h=len_h-len(dict_kfold['2']['Test'])
  dict_kfold ['1']= {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)}



  # Model architecture 
  #MODEL= model_1
  #MODEL_LABEL='M1'
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]={}
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Train']=[]
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Val']=[]
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Test']=[]

  #TRAINING MODEL
  cvscores_val = []
  cvscores_test=[]
  iter=0


  Train_RMSE=[]
  Train_MAE=[]
  Train_MAPE=[]
  Val_RMSE=[]
  Val_MAE=[]
  Val_MAPE=[]
  Test_RMSE=[]
  Test_MAE=[]
  Test_MAPE=[]


  for k, v in dict_kfold.items():



    #k='4'
    #v=dict_kfold[k]
    train_X, train_y= X[v['Train'],:,:], y[v['Train'],:,:]
    val_X, val_y=  X[v['Val'],:,:], y[v['Val'],:,:]
    test_X, test_y =  X[v['Test'],:,:], y[v['Test'],:,:]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], N_LAGS, N_FEATURES))
    val_X = val_X.reshape((val_X.shape[0], N_LAGS, N_FEATURES))
    test_X = test_X.reshape((test_X.shape[0], N_LAGS, N_FEATURES))
    print('Shape train set (X, y): ', train_X.shape, train_y.shape)
    print('Shape val set (X, y): ', val_X.shape, val_y.shape)
    print('Shape test set (X, y): ', test_X.shape, test_y.shape)

    # fit network
    #history = model.fit(train_X.astype('float32'), train_y.astype('float32'), epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_X.astype('float32'), val_y.astype('float32')), verbose=2, shuffle=False)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=PATIENCE,
                                                      mode='min') 
    if iter==0:
      model=MODEL((train_X.shape[1], train_X.shape[2]),PH)    
    model.compile(loss=tf.losses.MeanSquaredError(),
                    optimizer=tf.optimizers.Adam(),
                    metrics=[tf.metrics.MeanAbsoluteError()])

    history = model.fit(train_X.astype('float32'), train_y.astype('float32'), 
                          epochs=MAX_EPOCHS, 
                          batch_size=BATCH_SIZE, 
                          validation_data=(val_X.astype('float32'), val_y.astype('float32')), 
                          #verbose=2, shuffle=False,
                          callbacks=[early_stopping])



    # Generate generalization metrics
    scores_val= model.evaluate(val_X.astype('float32'), val_y.astype('float32'), verbose=1)
    scores_test = model.evaluate(test_X.astype('float32'), test_y.astype('float32'), verbose=1)

    print('Fold K (Val): ', (model.metrics_names[0], scores_val*100))
    print('Fold K (Test): ', (model.metrics_names[0], scores_test*100))

    cvscores_val.append(scores_val)
    cvscores_test.append(scores_test)

    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.legend()
    plt.show()


    ################################################################################
    ################################################################################
    #Training part
    # make a prediction
    yhat = model.predict(train_X)
    train_X = train_X.reshape((train_X.shape[0], train_X.shape[1]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, train_X), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,:PH]
    # invert scaling for actual
    train_y = train_y.reshape((len(train_y), PH))
    inv_y = np.concatenate((train_y, train_X), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,:PH]
    # calculate RMSE
    rmse = mean_squared_error(inv_y, inv_yhat)
    mae_ = mean_absolute_error(inv_y, inv_yhat)
    mape_=mean_absolute_percentage_error(inv_y, inv_yhat)

    print('Train RMSE:', rmse)
    print('Train MAE: ', mae_)
    print('Train MAPE(%): ', mape_)
    Train_RMSE.append(rmse)
    Train_MAE.append(mae_)
    Train_MAPE.append(mape_)


    plt.figure(figsize=(25,8))
    plt. plot(inv_y[:-1], label='Train')
    plt.plot(inv_yhat[1:], label= 'Estimated')
    plt.legend()
    plt.show()


    #Valdidation part
    # make a prediction
    yhat = model.predict(val_X)
    val_X = val_X.reshape((val_X.shape[0], val_X.shape[1]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, val_X), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,:PH]
    # invert scaling for actual
    val_y = val_y.reshape((len(val_y), PH))
    inv_y = np.concatenate((val_y, val_X), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,:PH]
    # calculate RMSE
    #rmse = mean_squared_error(inv_y[:-1], inv_yhat[1:])
    #mae_ = mean_absolute_error(inv_y[:-1], inv_yhat[1:])
    #mape_=mean_absolute_percentage_error(inv_y[:-1], inv_yhat[1:])

    rmse = mean_squared_error(inv_y, inv_yhat)
    mae_ = mean_absolute_error(inv_y, inv_yhat)
    mape_=mean_absolute_percentage_error(inv_y, inv_yhat)

    print('Val RMSE: ',rmse)
    print('Val MAE: ', mae_)
    print('Val MAPE(%): ', mape_)
    Val_RMSE.append(rmse)
    Val_MAE.append(mae_)
    Val_MAPE.append(mape_)

    plt.figure()
    plt. plot(inv_y[:-1], label='Val')
    plt.plot(inv_yhat[1:], label= 'Estimated')
    plt.legend()
    plt.show()



    ################################################################################
    ################################################################################

    #Test part
    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, test_X), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,:PH]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), PH))
    inv_y = np.concatenate((test_y, test_X), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,:PH]
    # calculate RMSE
    #rmse = mean_squared_error(inv_y[:-1], inv_yhat[1:])
    #mae_ = mean_absolute_error(inv_y[:-1], inv_yhat[1:])
    #mape_=mean_absolute_percentage_error(inv_y[:-1], inv_yhat[1:])

    rmse = mean_squared_error(inv_y, inv_yhat)
    mae_ = mean_absolute_error(inv_y, inv_yhat)
    mape_=mean_absolute_percentage_error(inv_y, inv_yhat)

    print('Test RMSE: ',rmse)
    print('Test MAE: ', mae_)
    print('Test MAPE(%): ', mape_)
    Test_RMSE.append(rmse)
    Test_MAE.append(mae_)
    Test_MAPE.append(mape_)

    plt.figure()
    plt. plot(inv_y[:-1], label='Test')
    plt.plot(inv_yhat[1:], label= 'Prediction')
    plt.legend()
    plt.show()

  # Save model
  model.save_weights(PATH+'weights_multi'+MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)+'.h5')
  
  # Save metrics
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]={}
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Train']=(Train_RMSE, Train_MAE,Train_MAPE)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Val']=(Val_RMSE, Val_MAE,Val_MAPE)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Test']=(Test_RMSE, Test_MAE,Test_MAPE)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Theil']= theil (inv_y, inv_yhat, test_X,PH)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Pocid']= pocid(inv_yhat,inv_y)

  pickle_out = open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle","wb")
  pickle.dump(res_barrido, pickle_out)
  pickle_out.close()






In [ ]:
#for PH in LIST_PH:
#  for N_LAGS in LIST_N_LAGS:
#    MODEL= model_1
#    MODEL_LABEL='M1'
#    with open(PATH+"pickle_res_barrido.pickle", "rb") as input_file:
#      res_barrido = pickle.load(input_file)
#    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)


### Model 1


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_1
  MODEL_LABEL='M1'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 2


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_2
  MODEL_LABEL='M2'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 4


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_4
  MODEL_LABEL='M4'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 5


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_5
  MODEL_LABEL='M5'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 6


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_6
  MODEL_LABEL='M6'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 3


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_7
  MODEL_LABEL='M7'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 10


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_10
  MODEL_LABEL='M10'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 11


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_11
  MODEL_LABEL='M11'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 12


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_12
  MODEL_LABEL='M12'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

### Model 13


In [ ]:
PH=3
for N_LAGS in LIST_N_LAGS:
  MODEL= model_13
  MODEL_LABEL='M13'
  print(MODEL_LABEL, str(N_LAGS))
  with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
    res_barrido = pickle.load(input_file)
  try:
    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)
  except:
    print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

# Barrido N_LAGS modelos con regularización

In [ ]:
## BARRIDO DE PARÁMETROS

LIST_PH=[1, 3,7,15]
LIST_N_LAGS=[180,90,30,15]

# PREDICTION PARAMS (no- change!)
#PH= 3
#N_LAGS= 180#30
MAX_EPOCHS = 20
BATCH_SIZE=32
PATIENCE=2
OPTIMIZER='adam'
lOSS_METRIC='mae'
from keras.regularizers import L1L2
REGULARIZERS = [L1L2(l1=0.01, l2=0.0), L1L2(l1=0.0, l2=0.01), L1L2(l1=0.01, l2=0.01)]
	

In [ ]:
### Using Stacked LSTM
## model 5
# define the model
def model_5(input_shape,PH,reg):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(70, return_sequences=True,input_shape=input_shape, recurrent_regularizer=reg))
 model.add(LSTM(30, return_sequences=False, recurrent_regularizer=reg))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

### Using Stacked LSTM
## model 6
# define the model
def model_6(input_shape,PH,reg):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(100, return_sequences=True,input_shape=input_shape, recurrent_regularizer=reg))
 model.add(LSTM(50, return_sequences=False, recurrent_regularizer=reg))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

 ### Using Stacked LSTM
## model 7
# define the model
def model_7(input_shape,PH,reg):
 dropout = 0.1
 model = Sequential()
 model.add(LSTM(70, return_sequences=True,input_shape=input_shape, recurrent_regularizer=reg))
 model.add(LSTM(70, return_sequences=False, recurrent_regularizer=reg))
 model.add(Dense(PH))
 model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
 return model

 ### Using Stacked LSTM
## model 10
# define the model
def model_10(input_shape,PH,reg):
  model = Sequential()
  model.add(LSTM(64, return_sequences=True,input_shape=input_shape, recurrent_regularizer=reg))
  model.add(LSTM(64, return_sequences=False, recurrent_regularizer=reg))
  model.add(Dense(32))
  model.add(Dense(8))
  model.add(Dense(PH))
  model.compile(loss=lOSS_METRIC, optimizer=OPTIMIZER)
  return model

In [ ]:
#Cargar librerías
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf

In [ ]:
## mAIN PROCESS

def main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,reg):
  #Prepar data k-validation
  # PREDICTION PARAMS
  OUT_STEPS=PH
  num_features=N_FEATURES
  #

  #porcessed signal
  X, y = split_sequence(list(data_scaled), N_LAGS, PH)
  data_transf=series_to_supervised(data_scaled, n_in=N_LAGS, n_out=PH, dropnan=True)

  ## Prepare data for 4-fold validation (method 1)

  # division train/val/test
  len_h=len(X)
  dict_kfold={'4': {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)} }
  print(len_h)
  len_h=len_h-len(dict_kfold['4']['Test'])
  dict_kfold ['3']= {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)}
  print(len_h)
  len_h=len_h-len(dict_kfold['3']['Test'])
  dict_kfold['2']= {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)}
  print(len_h)
  len_h=len_h-len(dict_kfold['2']['Test'])
  dict_kfold ['1']= {'Train': np.arange(0, int(per_train*len_h)),
                    'Val': np.arange(int(per_train*len_h), int(per_train*len_h)+int(per_val*len_h)), 
                    'Test':np.arange(int(per_train*len_h)+int(per_val*len_h), len_h)}



  # Model architecture 
  #MODEL= model_1
  #MODEL_LABEL='M1'
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]={}
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Train']=[]
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Val']=[]
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Test']=[]

  #TRAINING MODEL
  cvscores_val = []
  cvscores_test=[]
  iter=0


  Train_RMSE=[]
  Train_MAE=[]
  Train_MAPE=[]
  Val_RMSE=[]
  Val_MAE=[]
  Val_MAPE=[]
  Test_RMSE=[]
  Test_MAE=[]
  Test_MAPE=[]


  for k, v in dict_kfold.items():



    #k='4'
    #v=dict_kfold[k]
    train_X, train_y= X[v['Train'],:,:], y[v['Train'],:,:]
    val_X, val_y=  X[v['Val'],:,:], y[v['Val'],:,:]
    test_X, test_y =  X[v['Test'],:,:], y[v['Test'],:,:]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], N_LAGS, N_FEATURES))
    val_X = val_X.reshape((val_X.shape[0], N_LAGS, N_FEATURES))
    test_X = test_X.reshape((test_X.shape[0], N_LAGS, N_FEATURES))
    print('Shape train set (X, y): ', train_X.shape, train_y.shape)
    print('Shape val set (X, y): ', val_X.shape, val_y.shape)
    print('Shape test set (X, y): ', test_X.shape, test_y.shape)

    # fit network
    #history = model.fit(train_X.astype('float32'), train_y.astype('float32'), epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_X.astype('float32'), val_y.astype('float32')), verbose=2, shuffle=False)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=PATIENCE,
                                                      mode='min') 
    if iter==0:
      model=MODEL((train_X.shape[1], train_X.shape[2]),PH,reg)    
    model.compile(loss=tf.losses.MeanSquaredError(),
                    optimizer=tf.optimizers.Adam(),
                    metrics=[tf.metrics.MeanAbsoluteError()])

    history = model.fit(train_X.astype('float32'), train_y.astype('float32'), 
                          epochs=MAX_EPOCHS, 
                          batch_size=BATCH_SIZE, 
                          validation_data=(val_X.astype('float32'), val_y.astype('float32')), 
                          #verbose=2, shuffle=False,
                          callbacks=[early_stopping])



    # Generate generalization metrics
    scores_val= model.evaluate(val_X.astype('float32'), val_y.astype('float32'), verbose=1)
    scores_test = model.evaluate(test_X.astype('float32'), test_y.astype('float32'), verbose=1)

    print('Fold K (Val): ', (model.metrics_names[0], scores_val*100))
    print('Fold K (Test): ', (model.metrics_names[0], scores_test*100))

    cvscores_val.append(scores_val)
    cvscores_test.append(scores_test)

    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.legend()
    plt.show()


    ################################################################################
    ################################################################################
    #Training part
    # make a prediction
    yhat = model.predict(train_X)
    train_X = train_X.reshape((train_X.shape[0], train_X.shape[1]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, train_X), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,:PH]
    # invert scaling for actual
    train_y = train_y.reshape((len(train_y), PH))
    inv_y = np.concatenate((train_y, train_X), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,:PH]
    # calculate RMSE
    rmse = mean_squared_error(inv_y, inv_yhat)
    mae_ = mean_absolute_error(inv_y, inv_yhat)
    mape_=mean_absolute_percentage_error(inv_y, inv_yhat)

    print('Train RMSE:', rmse)
    print('Train MAE: ', mae_)
    print('Train MAPE(%): ', mape_)
    Train_RMSE.append(rmse)
    Train_MAE.append(mae_)
    Train_MAPE.append(mape_)


    plt.figure(figsize=(25,8))
    plt. plot(inv_y[:-1], label='Train')
    plt.plot(inv_yhat[1:], label= 'Estimated')
    plt.legend()
    plt.show()


    #Valdidation part
    # make a prediction
    yhat = model.predict(val_X)
    val_X = val_X.reshape((val_X.shape[0], val_X.shape[1]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, val_X), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,:PH]
    # invert scaling for actual
    val_y = val_y.reshape((len(val_y), PH))
    inv_y = np.concatenate((val_y, val_X), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,:PH]
    # calculate RMSE
    #rmse = mean_squared_error(inv_y[:-1], inv_yhat[1:])
    #mae_ = mean_absolute_error(inv_y[:-1], inv_yhat[1:])
    #mape_=mean_absolute_percentage_error(inv_y[:-1], inv_yhat[1:])

    rmse = mean_squared_error(inv_y, inv_yhat)
    mae_ = mean_absolute_error(inv_y, inv_yhat)
    mape_=mean_absolute_percentage_error(inv_y, inv_yhat)

    print('Val RMSE: ',rmse)
    print('Val MAE: ', mae_)
    print('Val MAPE(%): ', mape_)
    Val_RMSE.append(rmse)
    Val_MAE.append(mae_)
    Val_MAPE.append(mape_)

    plt.figure()
    plt. plot(inv_y[:-1], label='Val')
    plt.plot(inv_yhat[1:], label= 'Estimated')
    plt.legend()
    plt.show()



    ################################################################################
    ################################################################################

    #Test part
    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, test_X), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,:PH]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), PH))
    inv_y = np.concatenate((test_y, test_X), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,:PH]
    # calculate RMSE
    #rmse = mean_squared_error(inv_y[:-1], inv_yhat[1:])
    #mae_ = mean_absolute_error(inv_y[:-1], inv_yhat[1:])
    #mape_=mean_absolute_percentage_error(inv_y[:-1], inv_yhat[1:])

    rmse = mean_squared_error(inv_y, inv_yhat)
    mae_ = mean_absolute_error(inv_y, inv_yhat)
    mape_=mean_absolute_percentage_error(inv_y, inv_yhat)

    print('Test RMSE: ',rmse)
    print('Test MAE: ', mae_)
    print('Test MAPE(%): ', mape_)
    Test_RMSE.append(rmse)
    Test_MAE.append(mae_)
    Test_MAPE.append(mape_)

    plt.figure()
    plt. plot(inv_y[:-1], label='Test')
    plt.plot(inv_yhat[1:], label= 'Prediction')
    plt.legend()
    plt.show()

  # Save model
  model.save_weights(PATH+'weights_multi'+MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)+'.h5')
  
  # Save metrics
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]={}
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Train']=(Train_RMSE, Train_MAE,Train_MAPE)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Val']=(Val_RMSE, Val_MAE,Val_MAPE)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Test']=(Test_RMSE, Test_MAE,Test_MAPE)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Theil']= theil (inv_y, inv_yhat, test_X,PH)
  res_barrido[MODEL_LABEL+'_'+str(PH)+'_'+str(N_LAGS)]['Pocid']= pocid(inv_yhat,inv_y)

  pickle_out = open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle","wb")
  pickle.dump(res_barrido, pickle_out)
  pickle_out.close()






In [ ]:
#for PH in LIST_PH:
#  for N_LAGS in LIST_N_LAGS:
#    MODEL= model_1
#    MODEL_LABEL='M1'
#    with open(PATH+"pickle_res_barrido.pickle", "rb") as input_file:
#      res_barrido = pickle.load(input_file)
#    main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS)


### Model 7


#### Reg L1

In [ ]:
for PH in LIST_PH:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_7
    MODEL_LABEL='M7_regL1'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[0])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

In [ ]:
res_barrido.keys()

#### Reg L2

In [ ]:
for PH in LIST_PH:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_7
    MODEL_LABEL='M7_regL2'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[1])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

In [ ]:
res_barrido.keys()

#### Reg L1L2

In [ ]:
for PH in LIST_PH:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_7
    MODEL_LABEL='M7_regL1L2'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[0])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

In [ ]:
res_barrido.keys()

In [ ]:
for PH in LIST_PH[-1:]:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_7
    MODEL_LABEL='M7_regL1L2'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[0])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))

In [ ]:
for PH in [7]:
  for N_LAGS in LIST_N_LAGS[1:]:
    MODEL= model_7
    MODEL_LABEL='M7_regL1L2'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[0])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))

### Model 6


#### Reg L1

In [ ]:
for PH in LIST_PH:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_6
    MODEL_LABEL='M6_regL1'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[0])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

In [ ]:
res_barrido.keys()

#### Reg L2

In [ ]:
for PH in LIST_PH:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_6
    MODEL_LABEL='M6_regL2'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[1])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

In [ ]:
res_barrido.keys()

#### Reg L1L2

In [ ]:
for PH in LIST_PH:
  for N_LAGS in LIST_N_LAGS:
    MODEL= model_6
    MODEL_LABEL='M6_regL1L2'
    print(MODEL_LABEL, str(N_LAGS))
    with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
      res_barrido = pickle.load(input_file)
    try:
      main_process(data_scaled, res_barrido, MODEL, MODEL_LABEL,PH,N_LAGS,REGULARIZERS[0])
    except:
      print('Error in: ', MODEL_LABEL, str(N_LAGS))






In [ ]:
with open(PATH+"Definitive_scripts/"+"pickle_res_barrido.pickle", "rb") as input_file:
  res_barrido = pickle.load(input_file)

res_barrido

In [ ]:
res_barrido.keys()